# Esercitazione Solar Panel

## import librerie

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib as ptlt
import plotly.express as px
%matplotlib inline
import plotly.graph_objs as go

## lettura da csv

In [2]:
df_solar_panel=pd.read_csv("solar_panels.csv",sep=";")
df_solar_panel["DaySum"]=pd.to_numeric(df_solar_panel["DaySum"])

# Es 1

Analisi della produzione energetica
1) Determinare e graficare la produzione energetica totale (kWh) raggruppata nei seguenti intervalli temporali:
* Giornaliera
* Settimanale
* Mensile
* Stagionale
* Totale annuale
2) Il consumo medio di una famiglia di 4 persone è di 2.7MWh annui, guardando il totale annuale l’impianto in
questione è sufficiente?
3) Sempre guardando il totale annuale e considerando un costo medio di 0.35€/kWh per l’energia acquistata da un
fornitore, quanti soldi vengono risparmiati? Se l’eventuale energia surplus venisse rivenduta a 0.05€/kWh, quanti
soldi verrebbero guadagnati?
4) Assumendo che il consumo energetico sia ogni giorno lo stesso (dividere il totale per 365), quanti giorni l’impianto
ha fornito energia sufficiente a coprire il fabbisogno? Bonus: graficare il guadagno cumulato giorno per giorno
dovuto alla rivendita dell’energia in eccesso.
5) Si consideri ora che il proprietario possieda due Tesla Model S che percorrono all’anno 15000km ciascuna, con un
consumo di 180Wh al kilometro e ricaricate sempre a casa. Ripetere i punti 2), 3) e 4) considerando il fabbisogno
delle auto elettriche.

## 1) Determinare e graficare la produzione energetica totale (kWh) raggruppata in intervalli temporali

### produzione energetica giornaliera

In [13]:
df_kwh_giorno=df_solar_panel.set_index("Date")
df_kwh_giorno.index=pd.to_datetime(df_kwh_giorno.index)
df_kwh_giorno=pd.DataFrame(df_kwh_giorno.resample("D").max()["DaySum"]/1000)
len(df_kwh_giorno)

365

In [14]:
px.bar(df_kwh_giorno,
       y="DaySum",
       template='plotly_white',
       labels={"DaySum":"Somma kW/h","index":"Tempo (giorni)"},
       title="Produzione energetica totale giornaliera")

### produzione energetica settimanale

In [31]:
df_kwh_settimana=df_kwh_giorno.resample("W").sum()

In [17]:
fig = px.bar(df_kwh_settimana,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","week":"Settimane"},
             color_discrete_sequence=["green"],
             title="Produzione energetica totale settimanale")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in df_kwh_settimana.index]
    )
)
fig.show()

### produzione energetica mensile

In [34]:
df_kwh_mese = df_kwh_giorno.resample("M").sum()

In [19]:
fig = px.bar(df_kwh_mese,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","Date":"Mesi"},
             color_discrete_sequence=["orange"],
             title="Produzione energetica totale mensile")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in df_kwh_mese.index],
        ticktext = ["Gen","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    )
)
fig.show()

### produzione energetica stagionale e totale

In [21]:
df_kwh_stagioni = pd.DataFrame(index=["Primavera","Estate","Autunno","Inverno"],columns=["DaySum"]).fillna(0)
inizio_primavera = pd.to_datetime("2015-03-21")
inizio_estate = pd.to_datetime("2015-06-21")
inizio_autunno = pd.to_datetime("2015-09-22")
inizio_inverno = pd.to_datetime("2015-12-22")

In [38]:
kwh_totale = df_kwh_giorno.sum().squeeze()

In [24]:
df_kwh_stagioni.loc["Primavera","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.index>=inizio_primavera) & (df_kwh_giorno.index<inizio_estate)].DaySum)
df_kwh_stagioni.loc["Estate","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.index>=inizio_estate) & (df_kwh_giorno.index<inizio_autunno)].DaySum)
df_kwh_stagioni.loc["Autunno","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.index>=inizio_autunno) & (df_kwh_giorno.index<inizio_inverno)].DaySum)
df_kwh_stagioni.loc["Inverno","DaySum"] = kwh_totale - sum(df_kwh_stagioni.DaySum)

In [25]:
fig = px.bar(df_kwh_stagioni,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","index":"Stagioni"},
             color_discrete_sequence=["purple"],
             title="Produzione energetica totale stagionale",
             width=700,
             height=400,)
for data in fig.data:
    data["width"] =0.5
fig.show()

In [26]:
fig = px.bar(df_kwh_stagioni.transpose(),
             template='plotly_white',
             labels={"value":"kW/H","index":"", "variable":"Stagioni"},
             color_discrete_sequence= px.colors.qualitative.Plotly,
             title="Produzione energetica totale",
             width=600,
             height=500,
             barmode = "stack")
fig.update_xaxes(showticklabels=False)
for data in fig.data:
    data["width"] =0.4
fig.show()

In [27]:
px.pie(df_kwh_stagioni,  
        values='DaySum', 
        names=df_kwh_stagioni.index,
        title=f"Influenza delle stagioni sull'energia consumata ({int(kwh_totale)} kw/H)",
        width=600,
        height=500,)

## 2) Soldi risparmiati e soldi guadagnati

### risparmio energetico

In [43]:
risparmio = kwh_totale * 0.35
risparmio = round(risparmio, 2)
risparmio

2821.73

### guadagno dall'energia in eccesso

In [44]:
guadagno = (kwh_totale - 2700) * 0.05
guadagno = round(guadagno, 2)
guadagno

268.1

## 4) Quando l'impianto riesce a soddisfare il fabbisogno della famiglia?

In [45]:
fabb_med_giornaliero = 2700 /365
fig = px.bar(df_kwh_giorno,
       y="DaySum",
       template='plotly_white',
       labels={"DaySum":"Somma kW/h","index":"Tempo (giorni)"},
       title="Produzione energetica totale giornaliera")
fig.add_hline(y=fabb_med_giornaliero,opacity=0.5)
fig.show()

### grafico comulativo

In [46]:
df_guadagno_giornaliero = df_kwh_giorno.copy()
df_guadagno_giornaliero["Energia_Guadagnata"] = (df_guadagno_giornaliero["DaySum"] - fabb_med_giornaliero)

In [47]:
df_guadagno_giornaliero["Guadagno"]=0
df_guadagno_giornaliero.loc[df_guadagno_giornaliero["Energia_Guadagnata"] > 0, "Guadagno"] = df_guadagno_giornaliero["Energia_Guadagnata"]*0.05

df_guadagno_giornaliero["Spesa"]=0
df_guadagno_giornaliero.loc[df_guadagno_giornaliero["Energia_Guadagnata"] < 0, "Spesa"] = df_guadagno_giornaliero["Energia_Guadagnata"]*-0.35

df_guadagno_giornaliero["Ricavo"] = df_guadagno_giornaliero["Guadagno"] - df_guadagno_giornaliero["Spesa"]


df_guadagno_giornaliero["Color"] = ["Green" if x>0 else "Red" for x in df_guadagno_giornaliero["Ricavo"]]
df_guadagno_giornaliero

,DaySum,Energia_Guadagnata,Guadagno,Spesa,Ricavo,Color
Date,,,,,,
2015-01-01,12.204,4.80674,0.240337,0.000000,0.240337,Green
2015-01-02,11.447,4.04974,0.202487,0.000000,0.202487,Green
2015-01-03,18.222,10.82474,0.541237,0.000000,0.541237,Green
2015-01-04,37.805,30.40774,1.520387,0.000000,1.520387,Green
2015-01-05,5.912,-1.48526,0.000000,0.519841,-0.519841,Red
...,...,...,...,...,...,...
2015-12-27,1.727,-5.67026,0.000000,1.984591,-1.984591,Red
2015-12-28,2.495,-4.90226,0.000000,1.715791,-1.715791,Red
2015-12-29,3.985,-3.41226,0.000000,1.194291,-1.194291,Red


In [48]:
num_giorni_guadagno = len(df_guadagno_giornaliero[df_guadagno_giornaliero["Guadagno"] > 0])
num_giorni_guadagno

308

In [49]:
df_guadagno_giornaliero_cumulativo = pd.DataFrame(df_guadagno_giornaliero["Guadagno"].cumsum())
fig = px.line(df_guadagno_giornaliero_cumulativo,
        template='plotly_white',
        labels={"value":"Guadagno (€)","Date":"Tempo"},
        title="Guadagno cumulativo giornaliero",
        width=1000,
        height=700)

fig.update_layout(showlegend=False)
fig.show()

In [50]:
fig = px.bar(df_guadagno_giornaliero, y = "Ricavo", 
     template='plotly_white',
       labels={"index":"Tempo (giorni)","Ricavo":"Ricavo (€)"},
       title="Ricavo giornaliero",
       color = "Color",color_discrete_sequence=df_guadagno_giornaliero.Color.unique(),
       width=1400,
       height=450)

fig.update_layout(showlegend=False)
fig.show()

## 5) ora il proprietario ha 2 Tesla, ricalcola tutto

In [58]:
km_percorsi=15000*2
consumo_W_h_al_KM=180